# Cleaning Up the Ultra-Trail du Mont-Blanc (UTMB) Dataset
The inspiration for using this dataset came from the following Kaggle datasets:
- [Dataset: Ultra-Trail du Mont-Blanc 2003-2017](https://www.kaggle.com/ceruleansea/ultratrail-du-montblanc-20032017)

The original dataset taken from here: <https://utmbmontblanc.com/en/page/349/results.html>

## Importing Packages

In [1]:
import numpy as np
import pandas as pd

## Deffining Parameters

In [2]:
years = [2004, 2005, 2006, 2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
age_group_mapping = {
    'ES': 0,
    'SE': 1,
    'V1': 2,
    'V2': 3,
    'V3': 4,
    'V4': 5,
    'V5': 6,
}

## Downloading Dataset

Manually download the web pages containing the dataset as a table. Go to <https://utmbmontblanc.com/en/page/349/results.html> select each of the years, right click the web page and select "save as...". Save the pages in "../original/utmb_pages/{year}.html"

## Loading the Datset

In [3]:
datasets_list = []
for year in years:
    dataset_tmp = pd.read_html('../original/utmb_pages/{}.html'.format(year), attrs = {'id': 'tabPass'})[0]
    dataset_tmp = pd.DataFrame({'Rank': dataset_tmp[0],
                                'Name': dataset_tmp[2],
                                'Category': dataset_tmp[3],
                                'Year': year,
                                'Time': dataset_tmp[len(dataset_tmp.columns) - 1],
                                })
    dataset_tmp = dataset_tmp.dropna(axis=0)
    datasets_list.append(dataset_tmp)

full_dataset = pd.concat(datasets_list, axis=0, ignore_index=True)

## Displaying the first 10 rows of the dataset

In [4]:
print(len(full_dataset))
full_dataset.head(10)

19516


,Category,Name,Rank,Time,Year
0,SE H,DELEBARRE Vincent,1,21:06:18,2004
1,SE H,SHERPA Dachhiri-Dawa,2,23:02:28,2004
2,SE H,PACHE Jean Claude,3,23:40:08,2004
3,SE H,FAVRE-FELIX Lionel,4,23:50:36,2004
4,SE H,NEMETH Csaba,5,23:54:09,2004
5,SE H,CERCUEIL Michel,6,24:46:26,2004
6,V1 H,VERDIER Philippe,7,24:46:28,2004
7,V1 H,FATTON Christian,8,25:20:30,2004
8,V3 H,SCHWEIZER Werner,9,25:24:22,2004
9,V1 H,TECHER Henri,10,25:36:29,2004


## Cleaning up the data

In [5]:
dataset = full_dataset.copy()  # Creat a copy of the data

## convert time from string to float-hours
dataset['Result'] = pd.to_timedelta(dataset['Time']).astype(int) / 1e9 / 60 / 60
dataset['Age Group'] = dataset['Category'].apply(lambda x: age_group_mapping[x[:2]])
dataset['Gender'] = dataset['Category'].apply(lambda x: int(x[-1:] == 'F'))

## Remove unneccesery fields
dataset.pop('Category')
dataset.pop('Time')

## Print first 10 rows
dataset.head(10)

,Name,Rank,Year,Result,Age Group,Gender
0,DELEBARRE Vincent,1,2004,21.105000,1,0
1,SHERPA Dachhiri-Dawa,2,2004,23.041111,1,0
2,PACHE Jean Claude,3,2004,23.668889,1,0
3,FAVRE-FELIX Lionel,4,2004,23.843333,1,0
4,NEMETH Csaba,5,2004,23.902500,1,0
5,CERCUEIL Michel,6,2004,24.773889,1,0
6,VERDIER Philippe,7,2004,24.774444,2,0
7,FATTON Christian,8,2004,25.341667,2,0
8,SCHWEIZER Werner,9,2004,25.406111,4,0
9,TECHER Henri,10,2004,25.608056,2,0


## Save the clean dataset

In [6]:
dataset.to_csv('../../datasets/utmb.csv', index=False)